In [1]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd


Requiere : 
- beautifulsoup4
- lxml
- html5lib
- requests

In [2]:
# problemente esto requiera cambio, depeniendo del navegador y sistema operativo, no estoy seguro.

raw_header = """Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9
Accept-Encoding: gzip, deflate, br
Accept-Language: es-ES,es;q=0.9
Cache-Control: no-cache
Connection: keep-alive
Content-Length: 237
Content-Type: application/x-www-form-urlencoded
Cookie: _ga=GA1.2.908570269.1600309982; _gid=GA1.2.52737338.1600309982; _smart_data_session=ZzM5dmI2ZnlucW1sS0EveWc3UlhOaVBoRElMdDRwdVlTWm1oamc4aFVhakJ6Z2dtL0UvMGR6VUhxcjI4VEM4RERTdmdPU3hZZXJtak11b0tCR2tkYWJQbUVwempVVUlFN2JuVWxTVXo1UGMzc2pRa1FpWUV5K0o3ZzNGeVc2K2Z5ZnJRUHVkUGlEd1NWMlpjRmVNY09BPT0tLUFyc0tiMFdWS1diWFU2Wk1pU0JhTlE9PQ%3D%3D--8c25992c1ccbdacfc3f358d64da4841dab199ad1
Host: dataciencia.anid.cl
Origin: https://dataciencia.anid.cl
Pragma: no-cache
Referer: https://dataciencia.anid.cl/panorama/report
Sec-Fetch-Dest: document
Sec-Fetch-Mode: navigate
Sec-Fetch-Site: same-origin
Sec-Fetch-User: ?1
Upgrade-Insecure-Requests: 1
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36"""

In [3]:
headers_raw = {}
[headers_raw.update({header.split(":")[0]:header.split(":")[-1].strip()}) for header in raw_header.split("\n")];

In [4]:
json_headers = json.dumps(headers_raw,indent=4)
json_headers

'{\n    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",\n    "Accept-Encoding": "gzip, deflate, br",\n    "Accept-Language": "es-ES,es;q=0.9",\n    "Cache-Control": "no-cache",\n    "Connection": "keep-alive",\n    "Content-Length": "237",\n    "Content-Type": "application/x-www-form-urlencoded",\n    "Cookie": "_ga=GA1.2.908570269.1600309982; _gid=GA1.2.52737338.1600309982; _smart_data_session=ZzM5dmI2ZnlucW1sS0EveWc3UlhOaVBoRElMdDRwdVlTWm1oamc4aFVhakJ6Z2dtL0UvMGR6VUhxcjI4VEM4RERTdmdPU3hZZXJtak11b0tCR2tkYWJQbUVwempVVUlFN2JuVWxTVXo1UGMzc2pRa1FpWUV5K0o3ZzNGeVc2K2Z5ZnJRUHVkUGlEd1NWMlpjRmVNY09BPT0tLUFyc0tiMFdWS1diWFU2Wk1pU0JhTlE9PQ%3D%3D--8c25992c1ccbdacfc3f358d64da4841dab199ad1",\n    "Host": "dataciencia.anid.cl",\n    "Origin": "//dataciencia.anid.cl",\n    "Pragma": "no-cache",\n    "Referer": "//dataciencia.anid.cl/panorama/report",\n    "Sec-Fetch-Dest": "document",\n    "Sec-Fetch-M

## Generar POST , con data, el token y los headers
### TODO - ver como generar tokens, posiblemente expiren,no lo se. si no funciona hay que sacar otro del navegaor con la herramienta de inspeccion.



In [47]:
### 'BioBío', 'Ñuble', 'Maule','OHiggins', [8,9,10,16]

# regions[]: 8
# regions[]: 9
# regions[]: 10
# regions[]: 16
# from-year: 2008
# to-year: 2020
# commit: Generar

In [5]:
url = 'https://dataciencia.anid.cl/panorama/report'


token = "6cv6jfdkoKzGhXwFMIgYBso4xPkP+9DzlLpQx942EyThjNomG+vFdEMIrPPWQ5/xZh/K9iWKOCsmr3IDVK/5Ag=="
dict_data = {"authenticity_token":token,
             "regions[]":["8","9","10","16"],
             "commit":"Generar",
             "from-year":"2008",
             "to-year":"2020"
            }

In [60]:
r = requests.post(url,headers=headers_raw,data=dict_data)
print(r)

<Response [200]>


In [27]:
### Con GET se parsea todo, no disgregado por regiones

# url = 'https://dataciencia.anid.cl/panorama/report'

# r = requests.get(url)
# print(r)

<Response [200]>


In [61]:
soup = BeautifulSoup(r.text,"html.parser")

tables = soup.find_all("table",class_="table dataTable")


In [62]:
table_names = ["Disciplina OECD","Disciplina WoS","Institución","Región"]
datos_pd = {}
for i,t in enumerate(tables):
    pd_table = pd.read_html(t.prettify())[0]
    datos_pd[table_names[i]] = pd_table

## test iterativo

In [9]:
import time

In [8]:
años = [str(x) for x in range(2008,2021)]
regiones = [str(x) for x in range(1,17)]

In [6]:
regionSur=["8","9","10","16"]

In [11]:
y+"_"+r

'2008_9'

In [37]:
datos_años_region = {}
for y in años:
    for reg in regionSur:
        
        print("request año",y)
        print("request region",reg)
        dict_data.update({"from-year":y,"to-year":y,"regions[]":reg})

        r = requests.post(url,headers=headers_raw,data=dict_data)
        print("request completed",r.status_code)
        time.sleep(5)
        print("5 sec ready")
        soup = BeautifulSoup(r.text,"html.parser")
        tables = soup.find_all("table",class_="table dataTable")

        table_names = ["Disciplina OECD","Disciplina WoS","Institución","Región"]
        datos_pd = {}
        for i,t in enumerate(tables):
            pd_table = pd.read_html(t.prettify())[0]

            #add tag año
            pd_table["año"] = y
            pd_table["region"] = reg
            datos_pd[table_names[i]] = pd_table

        datos_años_region[y+"_"+reg] = datos_pd
        

request año 2008
request region 8
request completed 200
5 sec ready
request año 2008
request region 9
request completed 200
5 sec ready
request año 2008
request region 10
request completed 200
5 sec ready
request año 2008
request region 16
request completed 200
5 sec ready
request año 2009
request region 8
request completed 200
5 sec ready
request año 2009
request region 9
request completed 200
5 sec ready
request año 2009
request region 10
request completed 200
5 sec ready
request año 2009
request region 16
request completed 200
5 sec ready
request año 2010
request region 8
request completed 200
5 sec ready
request año 2010
request region 9
request completed 200
5 sec ready
request año 2010
request region 10
request completed 200
5 sec ready
request año 2010
request region 16
request completed 200
5 sec ready
request año 2011
request region 8
request completed 200
5 sec ready
request año 2011
request region 9
request completed 200
5 sec ready
request año 2011
request region 10
request

In [38]:
datos_años_region

{'2008_8': {'Disciplina OECD':                     Disciplina  # Publicaciones  # Autores % Hombre % Mujer  \
  0        AGRICULTURAL SCIENCES                2          3    100 %     0 %   
  1   ENGINEERING AND TECHNOLOGY                4          3    100 %     0 %   
  2  MEDICAL AND HEALTH SCIENCES                3          4    100 %     0 %   
  3             NATURAL SCIENCES                3          2    100 %     0 %   
  
     Producción Per Cápita   año region  
  0                    0.7  2008      8  
  1                    1.3  2008      8  
  2                    0.8  2008      8  
  3                    1.5  2008      8  ,
  'Disciplina WoS':                                      Disciplina  # Publicaciones  # Autores  \
  0                AGRICULTURE, MULTIDISCIPLINARY                1          2   
  1                                      AGRONOMY                1          2   
  2              CARDIAC & CARDIOVASCULAR SYSTEMS                2          4   
  3       

In [ ]:
# 'BioBío', 'Ñuble', 'Maule','OHiggins', [8,9,10,16]

In [39]:
map_region = {"16":"Ñuble","8":"O'Higgins","9":"Maule",
             "10":"Biobío"}

In [40]:
datos_region = []
datos_oecd = []
datos_wos = []
datos_inst = []
for a,d in datos_años_region.items():
    d["Región"]["region"] = d["Región"]["region"].map(map_region)
    d["Disciplina OECD"]["region"] = d["Disciplina OECD"]["region"].map(map_region)
    d["Disciplina WoS"]["region"] = d["Disciplina WoS"]["region"].map(map_region)
    d["Institución"]["region"] = d["Institución"]["region"].map(map_region)
    
    datos_region.append(d["Región"])
    datos_oecd.append(d["Disciplina OECD"])
    datos_wos.append(d["Disciplina WoS"])
    datos_inst.append(d["Institución"])
    

In [41]:
datos_region_conc = pd.concat(datos_region)
datos_oecd_conc = pd.concat(datos_oecd)
datos_wos_conc = pd.concat(datos_wos)
datos_inst_conc = pd.concat(datos_inst)

In [43]:
datos_region_conc.to_csv("data/DataCiencia_reporteRegion_sur.v2.csv")
datos_oecd_conc.to_csv("data/DataCiencia_reporteOECD_sur.v2.csv")
datos_wos_conc.to_csv("data/DataCiencia_reporteWoS_sur.v2.csv")
datos_inst_conc.to_csv("data/DataCiencia_reporteInst_sur.v2.csv")